<a href="https://colab.research.google.com/github/karsarobert/NLP2025/blob/main/02/NLP2024_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Természetes nyelvfeldolgozás
# PTE Gépi tanulás III.

## 1. Előadás: transzformerek, tokenizálók
### 2025. február 12.


**Bevezetés**

A mai órán a transzformátor modellek használatát fogjuk bemutatni a természetes nyelv feldolgozása (NLP) feladatokban. A transzformátor modellek egy mély tanulási modellcsalád, amelyeket 2017-ben a Google Research kutatói fejlesztettek ki. Azóta forradalmasították az NLP-t, és számos feladatban elérték a legmodernebb eredményeket.

**A transzformátor modellek architektúrája**

A transzformátor modellek egy "encoder" és egy "decoder" nevű két részből állnak. Az encoder a bemeneti szöveget egy vektorok sorozatává alakítja át, míg a decoder a vektorok sorozatát egy kimeneti szöveggé alakítja vissza. A transzformátor modellek "attention" mechanizmust használnak, amely lehetővé teszi a modell számára, hogy a bemeneti szöveg különböző részeire összpontosítson a kimeneti szöveg különböző részei generálásakor.

**A transzformátor modellek előnyei**

A transzformátor modellek számos előnnyel járnak a hagyományos NLP modellekkel szemben, mint amilyenek a rekurrens neurális hálózatok (RNN-ek):

* **Párhuzamosíthatóság:** A transzformátor modellek könnyen párhuzamosíthatók, ami lehetővé teszi a gyorsabb betanításukat és futtatásukat.
* **Hosszabb távú függőségek kezelése:** A transzformátor modellek jobban kezelik a hosszú távú függőségeket a bemeneti szövegben, mint az RNN-ek.
* **Jobb teljesítmény a legtöbb NLP feladatban:** A transzformátor modellek a legtöbb NLP feladatban elérték a legmodernebb eredményeket.

**Transzformátor modellek használata a Hugging Face Transformers-el**

A Hugging Face Transformers egy Python könyvtár, amely megkönnyíti a transzformátor modellek használatát NLP feladatokban. A könyvtár számos előre betanított transzformátor modellt is tartalmaz, amelyeket közvetlenül használhatunk.

**A mai órán a következőket fogjuk megtanulni:**

* Hogyan töltsünk be és mentsünk el egy transzformátor modellt a Hugging Face Transformers-szel.
* Hogyan használjunk egy transzformátor modellt szöveg osztályozására.
* Hogyan használjunk egy transzformátor modellt szöveg generálására.



Mi történik a pipeline függvényen belül?

A pipeline három szakaszból áll:

1. Tokenizálás:

A nyers szöveget tokenekre (kis egységekre) bontjuk.
Speciális tokeneket adunk hozzá (ha a modell megköveteli).
Minden tokent összekapcsolunk azonosítójával a modell szókincsében.
2. Modell:

Betöltjük a modell konfigurációját és súlyait.
A tokeneket bemenetként adjuk a modellnek.
A modell logitokat ad ki (nagy dimenziójú tenzor).
3. Feldolgozás utáni:

A logitokat valószínűségekké alakítjuk (SoftMax réteg).
Meghatározzuk a pozitív és negatív címkékhez tartozó valószínűségeket.

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.


In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier(
    [
        "I've been waiting for an NLP course my whole life.",
        "I hate this so much!",
    ]
)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


[{'label': 'NEGATIVE', 'score': 0.9893580079078674},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

![img](https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/full_nlp_pipeline.svg)

**Előfeldolgozás tokenizerrel**

Más neurális hálózatokhoz hasonlóan a Transformer modellek sem tudják közvetlenül feldolgozni a nyers szöveget, így a pipeline első lépése a szöveges bemenetek olyan számokká alakítása, amelyek értelmezhetőek a modell számára. Ehhez egy tokenizert használunk, amely a következőkért felelős:

* A bemenet szavakra, rész-szavakra vagy szimbólumokra (például írásjelekre) történő felosztása, amelyeket tokeneknek nevezünk.
* Minden tokent egy egész számhoz rendelünk.
* Olyan további bemenetek hozzáadása, amelyek hasznosak lehetnek a modell számára.

Mindezt az előfeldolgozást pontosan ugyanúgy kell elvégezni, mint ahogy a modell előzetes betanítása során történt. Ehhez először le kell töltenünk az információkat a Model Hub-ról.  Ennek érdekében az AutoTokenizer osztályt és annak from_pretrained() metódusát használjuk.  A modellünk checkpoint nevének használatával ez automatikusan lekéri a modell tokenizálóhoz kapcsolódó adatokat, és a gyorsítótárba helyezi őket (így csak akkor töltődik le, amikor először futtatod az alábbi kódot).

Mivel a sentiment-analysis pipeline alapértelmezett ellenőrzőpontja a distilbert-base-uncased-finetuned-sst-2-english (a modeladatlapot itt tekintheted meg), a következő kódot futtatjuk:


In [ ]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

**Miután rendelkezünk a tokenizerrel, közvetlenül átadhatjuk neki a mondatainkat, és visszakapunk egy szótárat, amely készen áll az adatok modellbe való betáplálásra! Az egyetlen még hátralévő teendő az input ID-k listájának tenzorokká alakítása.**

A 🤗 Transformers alkalmazását használhatod úgy, hogy nem kell aggódnod az ML keretrendszer miatt, ami a háttérben fut; lehet ez PyTorch, TensorFlow, vagy akár Flax bizonyos modellek esetében. Habár a Transformer modellek csak **tenzorokat** fogadnak el bemenetként. Ha még sosem hallottál tenzorokról, tekints rájuk úgy, mint NumPy tömbökre.  Egy NumPy tömb lehet egy skalár (0D), egy vektor (1D), egy mátrix (2D), vagy több dimenziós. Ez gyakorlatilag egy tenzor; más ML keretrendszerek tenzorjai hasonló módon viselkednek, és általában ugyanolyan egyszerűen példányosíthatók, mint a NumPy tömbök.

Annak meghatározásához, hogy milyen típusú tenzorokat szeretnénk visszakapni (PyTorch, TensorFlow vagy egyszerű NumPy), a return_tensors argumentumot használjuk:


In [ ]:
raw_inputs = [
    "I've been waiting for an NLP course my whole life.",
    "I hate this so much!",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs)

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  2019, 17953,  2361,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}


**Ne törődj még a paddinggel és a csonkítással (truncation); ezeket később fogjuk megnézni. A legfontosabb dolog, amit érdemes megjegyezni, hogy egyetlen mondatot, vagy mondatok listáját is átadhatsz, valamint meghatározhatod, hogy milyen típusú tenzorokat szeretnél visszakapni (ha nem adsz meg típust, eredményként listák listáját kapod).**

Így néznek ki az eredmények PyTorch tenzorként


**Maga a kimenet egy dict, szótár, amely két kulcsot tartalmaz: az `input_ids` és az `attention_mask`. Az  `input_ids` minden mondatból a tokenek egyedi azonosítóját tartalmazó egészek két sorát (mondatonként egyet) foglalja magában.


Az előre betanított modellünket ugyanúgy tölthetjük le, mint ahogy a tokenizert is tettük. A  🤗 Transformers rendelkezik egy `AutoModel` osztállyal, amelynek szintén van egy `from_pretrained()` metódusa:



In [ ]:
from transformers import AutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

**Ebben a kódrészletben ugyanazt a checkpoint-ot töltöttük le, amit korábban a pipeline folyamatunkban használtunk (ennek már a gyorsítótárban kellene lennie), és létrehoztunk egy modellt vele.**

Ez az architektúra csak az alap Transformer modult tartalmazza:  adott bemenetekre "rejtett állapotokat", más néven "tulajdonságokat" (features)  ad ki. Minden egyes modell inputra kapunk egy nagy dimenziós vektort, amely a Transformer modellnek az adott input kontextus szerinti megértését reprezentálja.

Ha ez most még nem világos, ne aggódj. Mindezt később elmagyarázzuk.

Ezek a rejtett állapotok önmagukban is hasznosak lehetnek, de általában egy másik modellrész, az úgynevezett "fej" (head) bemeneteiül szolgálnak.

**Miért nagy dimenziós a vektor?**

A Transformer modul által előállított vektor általában nagyméretű. Tipikusan három dimenziója van:

* **Batch size (kötegméret):** Az egyszerre feldolgozott szekvenciák száma (a példánkban 2).
* **Sequence length (szekvenciahossz):** A szekvencia numerikus reprezentációjának hossza (a példánkban 16).
* **Hidden size (rejtett méret):** Minden modell input vektorának dimenziója.

Azért nevezik "nagy dimenziójúnak" az utolsó dimenziószám miatt. A rejtett méret nagyon nagy lehet (a kisebb modelleknél gyakori a 768, a nagyobb modelleknél pedig akár 3072 vagy még nagyobb is lehet).

Ezt láthatjuk, ha beadjuk a modellünkbe azokat az inputokat, amelyeket már előre feldolgoztunk:


In [ ]:
outputs = model(**inputs) #(kettős csillag) operátor kibontja az inputs szótár tartalmát kulcs-érték párokká, és ezeket argumentumként adja át a model konstruktornak.
print(outputs.last_hidden_state.shape)
outputs

torch.Size([2, 16, 768])


BaseModelOutput(last_hidden_state=tensor([[[-0.1798,  0.2333,  0.6321,  ..., -0.3017,  0.5008,  0.1481],
         [ 0.2758,  0.6497,  0.3200,  ..., -0.0760,  0.5136,  0.1329],
         [ 0.9046,  0.0985,  0.2950,  ...,  0.3352, -0.1407, -0.6464],
         ...,
         [ 0.1466,  0.5661,  0.3235,  ..., -0.3376,  0.5100, -0.0561],
         [ 0.7500,  0.0487,  0.1738,  ...,  0.4684,  0.0030, -0.6084],
         [ 0.0519,  0.3729,  0.5223,  ...,  0.3584,  0.6500, -0.3883]],

        [[-0.2937,  0.7283, -0.1497,  ..., -0.1187, -1.0227, -0.0422],
         [-0.2206,  0.9384, -0.0951,  ..., -0.3643, -0.6605,  0.2407],
         [-0.1536,  0.8988, -0.0728,  ..., -0.2189, -0.8528,  0.0710],
         ...,
         [-0.3017,  0.9002, -0.0200,  ..., -0.1082, -0.8412, -0.0861],
         [-0.3338,  0.9674, -0.0729,  ..., -0.1952, -0.8181, -0.0634],
         [-0.3454,  0.8824, -0.0426,  ..., -0.0993, -0.8329, -0.1065]]],
       grad_fn=<NativeLayerNormBackward0>), hidden_states=None, attentions=None)

**Ne feledd, hogy a Transformers modellek kimenetei úgy viselkednek, mint névvel ellátott rendezett n-esek (namedtuples) vagy szótárak. Az elemekhez hozzáférhetsz attribútumokkal (ahogy az előbb tettük), vagy kulcs szerint (outputs["last_hidden_state"]), vagy akár index segítségével, ha pontosan tudod, hogy mit keresel, hol található (outputs[0]).**

**Modell fejek:  Értelmezzük a számokat**

A modell fejek a rejtett állapotok nagyméretű vektorát veszik bemenetként, és azt egy másik dimenzióba vetítik. Általában egy vagy néhány lineáris rétegből állnak:

![img](https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/transformer_and_head.svg)

**A Transformer modell kimenete közvetlenül a modell következő rétegének fejére kerül.**

Ebben a diagramban a modellt az embedding (beágyazó) réteg és az azt követő rétegek jelképezik.  Az embedding réteg minden egyes input azonosítót a tokenizált bemenetben átalakít egy vektorrá, ami a hozzá tartozó tokent reprezentálja. Az azt következő rétegek ezeket a vektorokat manipulálják a figyelemi mechanizmus használatával, hogy létrehozzák a mondatok végső reprezentációját.

**A Transformers által nyújtott különböző architektúrák**

A Transformers-ben számos különböző architektúra található, amelyek mindegyikét egy-egy meghatározott feladat kezelésére terveztek. Íme egy nem teljeskörű lista:

* Model (rejtett állapotok lehívása)
* ForCausalLM
* ForMaskedLM
* ForMultipleChoice
* ForQuestionAnswering
* ForSequenceClassification
* ForTokenClassification
* és más architektúrák

A saját példánkhoz szükségünk van egy olyan modellre, amely szekvencia osztályozó fejjel rendelkezik (hogy pozitívként vagy negatívként osztályozhassuk a mondatokat).  Így valójában  nem az  `AutoModel` osztályt fogjuk használni, hanem az `AutoModelForSequenceClassification` osztályt:


In [ ]:
from transformers import AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)

Ha most megnézzük a kimeneteink alakját, a dimenzionalitás sokkal kisebb lesz: a modellfej bemenetként a korábban látott nagydimenziós vektorokat veszi, és két értéket tartalmazó vektorokat ad ki (címkénként egyet):

In [ ]:
print(outputs.logits.shape)

torch.Size([2, 2])


A kimenet utófeldolgozása
A modellünk kimeneteként kapott értékeknek önmagukban nem feltétlenül van értelme. Vessünk rá egy pillantást:

In [ ]:
print(outputs.logits)

tensor([[-1.5607,  1.6123],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward0>)


Modellünk az első mondat esetében [-1,5607, 1,6123], a második esetében pedig [ 4,1692, -3,3464] értéket jelzett előre. Ezek nem valószínűségek, hanem logitek, a modell utolsó rétege által kiadott nyers, nem normalizált pontszámok. Ahhoz, hogy valószínűséggé alakíthassuk őket, át kell menniük egy SoftMax rétegen (minden Transformers modell logitokat ad ki, mivel a képzés veszteségfüggvénye általában az utolsó aktiválási függvényt, például a SoftMaxot, a tényleges veszteségfüggvénnyel, például a keresztentrópiával olvasztja össze):

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification


predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

tensor([[4.0195e-02, 9.5980e-01],
        [9.9946e-01, 5.4418e-04]], grad_fn=<SoftmaxBackward0>)


Most láthatjuk, hogy a modell az első mondat esetében [0,0402, 0,9598], a második esetében pedig [0,9995, 0,0005] értéket jósolt. Ezek felismerhető valószínűségi pontszámok.

Az egyes pozíciókhoz tartozó címkékhez az id2label attribútumot vizsgálhatjuk meg a modell konfigurációjában

In [ ]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}

Most már megállapíthatjuk, hogy a modell a következőket jósolta:

Az első mondat: NEGATÍV: 0,0402, POZITÍV: 0,9598.
Második mondat: NEGATÍV: 0,9995, POZITÍV: 0,0005
Sikeresen reprodukáltuk a csővezeték három lépését: előfeldolgozás tokenizátorokkal, a bemenetek átvezetése a modellen, és utófeldolgozás! Most szánjunk egy kis időt arra, hogy mélyebben elmerüljünk az egyes lépésekben.

✏️ Próbálja ki! Válasszon ki két (vagy több) saját szöveget, és futtassa le őket a mondatelemző csővezetéken. Ezután ismételd meg magad az itt látott lépéseket, és ellenőrizd, hogy ugyanazokat az eredményeket kapod-e!

In [ ]:
import torch
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"

model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
raw_inputs = [
    "Szép az idő.",
    "Utálom a matematikát",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
outputs = model(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

tensor([[0.7658, 0.2342],
        [0.8894, 0.1106]], grad_fn=<SoftmaxBackward0>)


Ebben a részben közelebbről megnézzük a modell létrehozását és használatát. Az AutoModel osztályt fogjuk használni, ami akkor praktikus, ha bármilyen modellt szeretnénk egy ellenőrzőpontból betölteni.

Ha azonban tudjuk, hogy milyen típusú modellt szeretnénk használni, akkor közvetlenül használhatjuk az architektúrát definiáló osztályt. Nézzük meg, hogyan működik ez egy BERT modellel.

Egy transzformátor létrehozása
A BERT modell inicializálásához először is be kell töltenünk egy konfigurációs objektumot:

In [ ]:
from transformers import BertConfig, BertModel

# Building the config
config = BertConfig()

# Building the model from the config
model = BertModel(config)

A konfiguráció számos attribútumot tartalmaz, amelyeket a modell felépítéséhez használnak:

In [ ]:
print(config)

BertConfig {
  "_attn_implementation_autoset": true,
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.48.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



Bár még nem láttad, hogy ezek az attribútumok mit csinálnak, néhányat fel kell ismerned: a hidden_size attribútum a hidden_states vektor méretét határozza meg, a num_hidden_layers pedig a Transformer modell rétegeinek számát.

##Különböző betöltési módszerek
Egy modell létrehozása az alapértelmezett konfigurációból véletlenszerű értékekkel inicializálja azt:

In [ ]:
from transformers import BertConfig, BertModel

config = BertConfig()
model = BertModel(config)

# Model is randomly initialized!

A modell ebben az állapotban is használható, de halandzsát fog kiadni; először be kell tanítani. A modellt a semmiből is betaníthatnánk az adott feladatra, de  ez hosszú időt és sok adatot igényelne, és nem elhanyagolható környezeti hatással járna. A felesleges és megkettőzött erőfeszítések elkerülése érdekében elengedhetetlen, hogy a már betanított modelleket megoszthassuk és újra felhasználhassuk.

##Egy már betanított Transformer modell betöltése egyszerű - ezt a from_pretrained() metódussal tehetjük meg:

In [ ]:
from transformers import BertModel

model = BertModel.from_pretrained("bert-base-cased")

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

A fenti kódpéldában nem használtuk a BertConfig-et, helyette egy előre betanított modellt töltöttünk be a bert-bázisú azonosítón keresztül. Ez egy olyan modell ellenőrzőpont, amelyet maguk a BERT szerzői képeztek ki; további részleteket a modellkártyáján találhat róla.

Ez a modell most az ellenőrzőpont összes súlyával inicializálódik. Közvetlenül használható következtetésre azokon a feladatokon, amelyekre betanították, és finomhangolható egy új feladaton is. Ha nem nulláról, hanem előre betanított súlyokkal képezzük, gyorsan jó eredményeket érhetünk el.

A súlyok letöltésre és gyorsítótárba kerültek (így a from_pretrained() metódus jövőbeli hívásai nem töltik le őket újra) a cache mappába, amely alapértelmezés szerint a ~/.cache/huggingface/transformers mappába kerül. A HF_HOME környezeti változó beállításával testre szabhatja a cache mappát.

A modell betöltéséhez használt azonosító lehet a Model Hubon lévő bármely modell azonosítója, amennyiben az kompatibilis a BERT architektúrával.

Mentési módszerek
A modell mentése ugyanolyan egyszerű, mint a betöltése - a save_pretrained() metódust használjuk, amely analóg a from_pretrained() metódussal:

In [ ]:
model.save_pretrained("directory_on_my_computer")

In [ ]:
!ls directory_on_my_computer

config.json  model.safetensors


Ha vet egy pillantást a config.json fájlra, felismeri a modellarchitektúra felépítéséhez szükséges attribútumokat. Ez a fájl tartalmaz néhány metaadatot is, például azt, hogy honnan származik az ellenőrzőpont, és hogy milyen Transformers verziót használtál, amikor utoljára mentetted az ellenőrzőpontot.

A pytorch_model.bin fájl az úgynevezett állapotszótár; ez tartalmazza a modell összes súlyát. A két fájl kéz a kézben jár; a konfiguráció a modelled felépítésének megismeréséhez szükséges, míg a modellsúlyok a modelled paraméterei.

A Transformer modell használata következtetéshez
Most, hogy már tudod, hogyan tölts be és ments el egy modellt, próbáljuk meg használni azt néhány előrejelzés elkészítéséhez. A Transformer modellek csak számokat tudnak feldolgozni - olyan számokat, amelyeket a tokenizáló generál. Mielőtt azonban a tokenizátorokról beszélnénk, vizsgáljuk meg, milyen bemeneteket fogad el a modell.

A tokenizátorok gondoskodnak a bemenetek megfelelő keretrendszer tenzoraiba való átkonvertálásáról, de hogy segítsünk megérteni, mi történik, gyorsan megnézzük, mit kell tenni, mielőtt a bemeneteket elküldjük a modellnek.

Tegyük fel, hogy van néhány szekvenciánk:

In [ ]:
sequences = ["Hello!", "Cool.", "Nice!"]

A tokenizáló ezeket szókincsindexekké alakítja át, amelyeket jellemzően bemeneti azonosítóknak nevezünk. Minden szekvencia most már egy számok listája! Az eredmény a következő kimenet:

In [ ]:
inputs = tokenizer(sequences, return_tensors="pt")
print(inputs['input_ids'])

tensor([[ 101, 7592,  999,  102],
        [ 101, 4658, 1012,  102],
        [ 101, 3835,  999,  102]])


In [ ]:
encoded_sequences = [
    [101, 7592, 999, 102],
    [101, 4658, 1012, 102],
    [101, 3835, 999, 102],
]

Ez a kódolt szekvenciák listája: listák listája. A tenzorok csak téglalap alakzatokat fogadnak el (gondoljunk a mátrixokra). Ez a "tömb" már téglalap alakú, így a tenzorrá alakítása egyszerű:

A tenzorok használata a modell bemeneteként
A tenzorok felhasználása a modellel rendkívül egyszerű - csak meghívjuk a modellt a bemenetekkel:

In [ ]:
import torch

model_inputs = torch.tensor(encoded_sequences)

In [ ]:
output = model(model_inputs)

Bár a modell számos különböző argumentumot fogad el, csak a bemeneti azonosítókra van szükség. Később elmagyarázzuk, hogy a többi argumentum mire szolgál, és mikor van rájuk szükség, de előbb közelebbről meg kell néznünk a tokenizátorokat, amelyekből a Transformer modell által értelmezhető bemenetek felépülnek.

In [ ]:
output
#output.last_hidden_state.shape

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 4.4496e-01,  4.8276e-01,  2.7797e-01,  ..., -5.4032e-02,
           3.9393e-01, -9.4770e-02],
         [ 2.4943e-01, -4.4093e-01,  8.1772e-01,  ..., -3.1917e-01,
           2.2992e-01, -4.1172e-02],
         [ 1.3668e-01,  2.2518e-01,  1.4502e-01,  ..., -4.6915e-02,
           2.8224e-01,  7.5566e-02],
         [ 1.1789e+00,  1.6738e-01, -1.8187e-01,  ...,  2.4671e-01,
           1.0441e+00, -6.1972e-03]],

        [[ 3.6436e-01,  3.2464e-02,  2.0258e-01,  ...,  6.0110e-02,
           3.2451e-01, -2.0996e-02],
         [ 7.1866e-01, -4.8725e-01,  5.1740e-01,  ..., -4.4012e-01,
           1.4553e-01, -3.7545e-02],
         [ 3.3223e-01, -2.3271e-01,  9.4876e-02,  ..., -2.5268e-01,
           3.2172e-01,  8.1085e-04],
         [ 1.2523e+00,  3.5754e-01, -5.1320e-02,  ..., -3.7840e-01,
           1.0526e+00, -5.6255e-01]],

        [[ 2.4042e-01,  1.4718e-01,  1.2110e-01,  ...,  7.6062e-02,
           3.3564e-01,  2

#Tokenizátorok
A tokenizátorok az NLP-csatorna egyik központi eleme. Egyetlen célt szolgálnak: a szöveget a modell által feldolgozható adatokká alakítják. A modellek csak számokat tudnak feldolgozni, ezért a tokenizátoroknak a szöveges bemeneteinket számadatokká kell alakítaniuk. Ebben a szakaszban megvizsgáljuk, hogy pontosan mi történik a tokenizáló csővezetékben.

Az NLP-feladatokban a feldolgozott adatok általában nyers szövegek. Íme egy példa ilyen szövegre:

Jim Henson bábszínész volt

A modellek azonban csak számokat tudnak feldolgozni, ezért meg kell találnunk a nyers szöveg számokká alakításának módját. Ezt teszik a tokenizátorok, és ennek számos módja van. A cél az, hogy megtaláljuk a legértelmesebb - vagyis a modell számára legértelmesebb - és lehetőleg a legkisebb reprezentációt.

Nézzünk meg néhány példát a tokenizáló algoritmusokra, és próbáljuk megválaszolni a tokenizálással kapcsolatos néhány kérdést.

Az első tokenizáló típus, ami eszünkbe jut, a szóalapú. Általában nagyon könnyen beállítható és használható, mindössze néhány szabály segítségével, és gyakran tisztességes eredményeket ad. Például az alábbi képen a cél a nyers szöveg szavakra bontása és számszerű ábrázolásuk megtalálása:

![img](https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/word_based_tokenization.svg)

A szöveg felosztásának különböző módjai vannak. Például használhatunk szóközöket a szöveg szavakra történő tokenizálására a Python split() függvényének alkalmazásával:

In [ ]:
tokenized_text = "Jim Henson was a puppeteer".split()
print(tokenized_text)

['Jim', 'Henson', 'was', 'a', 'puppeteer']


**A szó-tokenizereknek vannak olyan változatai is, amelyek az írásjeleket extra szabályokkal kezelik.  Ezzel a  tokenizáló-típussal végül igencsak nagy "szókincsekkel" találkozhatunk, ahol a szókincs alatt a korpuszunkban szereplő egymástól független tokenek teljes számát értjük.**

Minden szóhoz egy azonosító (ID) kerül  hozzárendelésre 0-val kezdve majd sorban fel  a szókincs nagyságáig. A modell ezeket az azonosítókat használja az egyes szavak beazonosítására.

**Hátrányok**

Ha egy teljes nyelvet le szeretnénk fedni  szó alapú tokenizálóval, rendelkeznünk kellene azonosítóval a nyelv minden egyes szavára, ami rengeteg tokent fog eredményezni. Például az angol nyelvben több mint 500 ezer szó van, tehát ha minden szóhoz tartozó inputazonsítóra egy saját térképet építenénk, annyi azonosítót kéne nyomon követnünk. Ezen kívül olyan szavak. pl. mint angolul a "dog" a  "dogs"-tól eltérően vannak ábrázolva, az elején a modellnek fogalma se lesz róla, hogy a  "dog" és a "dogs" hasonlóak: két egymással nem összefüggő szóként fogja  azonosítani őket. Ugyanez érvényes  más hasonló szavakra is: angolul mint például a "run" és a "running", amelyeket a modell eleinte nem érzékel majd hasonlókként.

Végül egy egyedi tokenre is szükségünk van olyan szavak  ábrázolására, amelyek nincsenek benne a szókincsünkben.  Ezt nevezzük "ismeretlen token"-nek, amit gyakran "[UNK]" vagy "<unk>" szimbólumok jelképeznek. Általában rossz jelnek számít, ha a tokenizáló sok ilyen tokent produkál, mert a szó értelmezhető reprezentációját nem tudta előállítani, így menet közben információt veszítünk. A cél a szókincs kidolgozásakor az, hogy olyan módon tegyük, hogy a tokoenizáló a lehető legkevesebb szót jelölje meg ismeretlen jelölővel.

Az ismeretlen tokenek számának csökkentésére szolgáló egyik módszer, ha egy szinttel lejjebb menünk, vagyis egy karakter alapú tokenizálót használunk.

**Karakter alapú**

*Karakter alapú* tokenizálók a szöveget szavak helyett karakterekre osztják szét. Ennek két elsődleges előnye van:

* Sokkal kisebb szókincsre van szükségünk.
* Sokkal kevesebb a szókincsen kívüli (ismeretlen) token, mivel minden szó felépíthető karakterekből.

Viszont itt is felmerülnek bizonyos kérdések a szóközöket és írásjeleket illetően:

![img](https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/character_based_tokenization.svg)

Ez a megközelítés sem tökéletes. Mivel az ábrázolás most már nem szavakon, hanem karaktereken alapul, azt mondhatjuk, hogy intuitíve kevésbé értelmes: minden egyes karakter önmagában nem jelent sokat, míg a szavak esetében ez a helyzet. Ez azonban megint csak nyelvenként változik; a kínai nyelvben például minden egyes karakter több információt hordoz, mint egy latin nyelvben egy karakter.

Egy másik dolog, amit figyelembe kell venni, hogy a végén nagyon nagy mennyiségű token lesz, amit a modellünknek fel kell dolgoznia: míg egy szó egy szóalapú tokenizálóval csak egyetlen token lenne, addig karakterekké alakítva könnyen 10 vagy több token lehet belőle.

Ahhoz, hogy mindkét világból a legjobbat kapjuk, használhatunk egy harmadik technikát, amely egyesíti a két megközelítést: az alszavas tokenizálást.


##Rész-szavas (subword) tokenizáció

A rész-szavas tokenizációs algoritmusok azon az elven alapulnak, hogy a gyakran használt szavakat nem szabad kisebb részekből álló szavakra felbontani, viszont a ritka szavakat érdemes értelmes részekre bontani.

Például az "annoyingly" az angolban ritka szónak számítana, és szét lehetne bontani az  "annoying" és  "ly" elemekre. Mindkettő valószínűleg nagyobb gyakorisággal fog előfordulni önálló rész-szavakként, miközben az  "annoyingly" jelentésének megőrzéséhez hozzájárulnak az "annoying" és "ly" jelentései együtt.

Egy példa arra, hogy a subword-tokenizáció hogyan bontaná fel a "Let's do tokenization!" szekvenciát:

![img](https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/bpe_subword.svg)



**Részszavas megoldások jelentősége**

Ezek a részszavak (subwords) rengeteg szemantikai jelentéssel bírnak: például a fentebb említett esetben a  "tokenization" fel lett bontva "token" és  "ization" jelentésű elemekre, ez a két token szemantikai jelentéshordozó, miközben helymegtakarító  ( mindössze két token szükséges egy hosszú szó ábrázolásához). Ez lehetővé teszi, hogy relatíve jó lefedettséggel  és kevés szókinccsel közel ismeretlen-mentes eredményt érjünk el.

Ez a megközelítés különösen hasznos az agglutinatív nyelvekben, például törökben, ahol tetszőlegesen hosszú összetett szavakat lehet képezni al-szavak összeláncolásával.

**Nem állunk meg itt**

Nem meglepő módon, sok más technika létezik a fentieken kívül. Csak hogy párat említsünk:

* Byte-szintű BPE, ahogy a GPT-2 használja
* WordPiece, ahogy a BERT használja
* SentencePiece vagy Unigram, ahogy több többnyelvű modell használja

A fenti alapismeretek elegendőek, hogy elkezdjünk dolgozni az API-val.

**Betöltés és mentés**

A tokenizerek betöltése és mentése épp olyan egyszerű, mint a modelleké. Igazából, ugyanazokon a módszereken alapul:
a `from_pretrained()` és a `save_pretrained()`. Ezek a metódusok betöltik/mentik a tokenizáló által használt algoritmust (hasonlóan a modell architektúrájához), valamint a szókincsét (hasonlóan a modell paramétereihez).  

A BERT tokenizáló betöltése, ami ugyanazzal a checkpointtal trenelt, mint a BERT, ugyanúgy történik, mint a modell betöltése. De itt a `BertTokenizer` osztályt használjuk az `AutoTokenizer` helyett:


In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Az AutoModel-hez hasonlóan az AutoTokenizer osztály is megragadja a megfelelő tokenizáló osztályt a könyvtárban az ellenőrzőpont neve alapján, és közvetlenül használható bármely ellenőrzőponttal:

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [ ]:
tokenizer("Using a Transformer network is simple")

{'input_ids': [101, 7993, 170, 13809, 23763, 2443, 1110, 3014, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokenizer.save_pretrained("directory_on_my_computer")

('directory_on_my_computer/tokenizer_config.json',
 'directory_on_my_computer/special_tokens_map.json',
 'directory_on_my_computer/vocab.txt',
 'directory_on_my_computer/added_tokens.json',
 'directory_on_my_computer/tokenizer.json')

##Szöveg számokká alakítása

Ez az eljárás: **kódolás (encoding)**néven ismert.  A kódolás egy kétlépcsős folyamat:

Ahogy azt már láttuk, az első lépés a szöveg szavakra (vagy szórészekre,  írásjelekre stb.)  bontása, amit általában tokeneknek nevezünk. Számos szabály vonatkozhat a folyamatra, ezért a tokenizáló példányosításakor a modell nevét kell használni annak érdekében, hogy biztosan ugyanazon szabályokat alkalmazzuk, amelyeket a modell pretrainingje során használtak.

A második lépés az, hogy ezeket a tokeneket számokká alakítjuk, hogy tenzort készíthessünk belőlük, és megadhassuk őket a modellnek. Ehhez a tokenizáló rendelkezik egy **szókinccsel**, ez az a rész, amit letöltünk, amikor létrehozzuk a  `from_pretrained()` metódussal. Ismét a modell előzetes betanítása során használt szókincset kell használnunk.

E két lépés mélyebb megismerése érdekében külön-külön fogjuk kielemezni azokat. Fontos megjegyzés, hogy az elemzés során bizonyos módszereket  fogunk használni, amelyek elvégzik a tokenizáló pipeline egyes elemeit külön-külön mutatva az adott lépések közbenső eredményeit.

**Tokeinizáció**

A  **tokenize()**  metódus hajtja végre a tokenizációs folyamatot a tokenizálóban:


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

sequence = "Using a Transformer network is simple"
tokens = tokenizer.tokenize(sequence)

print(tokens)

['Using', 'a', 'Trans', '##former', 'network', 'is', 'simple']


Ennek a módszernek a kimenete egy karakterláncok vagy tokenek listája:

In [ ]:
ids = tokenizer.convert_tokens_to_ids(tokens)

print(ids)

[7993, 170, 13809, 23763, 2443, 1110, 3014]


**Tensorok, mint input**

Ezeket a kimeneteket, miután átalakítottuk őket a megfelelő keretrendszer tenzorává, a modell bemeneteként használhatjuk, ahogy azt ebben a fejezetben már korábban láttuk.

✏️ **Próbáld ki!** Hajtsd végre az utolsó két lépést (tokenizáció és konvertálás inputazonsítókká)  a 2. fejezetben használt input kifejezéseken ("I've been waiting for a Hugging Face course my whole life." and "I hate this so much!"). Ellenőrizd, hogy ugyanazokat az ids-eket kapod-e, amiket korábban!




In [ ]:
sequence = "I've been waiting for an NLP course my whole life."
tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

[146, 112, 1396, 1151, 2613, 1111, 170, 20164, 10932, 2271, 7954, 1736, 1139, 2006, 1297, 119]


**Dekódolás**

A dekódolás a fordítottja a tokenezésnek : a szókincs indexei alapján (vissza)kapjuk a karakterláncot. Ez a következőképpen hajtható végre a `decode()` metódussal:

In [ ]:
decoded_string = tokenizer.decode([7993, 170, 11303, 1200, 2443, 1110, 3014])
print(decoded_string)

Using a transformer network is simple


Vegyük észre, hogy a dekódolási módszer nemcsak az indexeket alakítja vissza tokenekké, hanem az azonos szavak részét képező tokeneket is csoportosítja, hogy olvasható mondatot kapjunk. Ez a viselkedés rendkívül hasznos lesz, amikor olyan modelleket használunk, amelyek új szöveget jósolnak meg (akár egy promptból generált szöveget, akár szekvencia-szekvencia problémákat, például fordítást vagy összegzést).

Mostanra már meg kell értenie azokat az atomi műveleteket, amelyeket egy tokenizáló kezelni tud: tokenizálás, azonosítókká alakítás és az azonosítók visszaalakítása karakterlánccá.

#Több szekvencia kezelése

**Új felmerülő kérdések**

Az előző részben a legegyszerűbb alkalmazási esetet vettük át: következtetés lefuttatását egyetlen szekvencián.  Viszont az alábbi kérdések merülnek fel:

* Hogyan kezeljük egyszerre több szekvenciát?
* Hogyan kezeljük egyszerre több eltérő hosszúságú szekvenciát?
* A szókincs-indexeken kívül vannak más input-típusok a modell optimális működéséhez?
* Létezik túl hosszú szekvencia?

Lássuk, milyen jellegű problémák merülhetnek fel, és hogyan oldhatók ezek meg a Transformers API használatával.

**A modellek bemenetként egy "köteget" várnak**

Az előző  feladat során láttad, hogyan konvertálhatók a szekvenciák számok listájává.  Ezek után a számok listáját átalakítjuk egy tenzorrá, és ezzel beadhatjuk a bemenetet a modellnek:


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for an NLP course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
input_ids = torch.tensor(ids)
# This line will fail.
model(input_ids)

IndexError: too many indices for tensor of dimension 1

In [ ]:
input_ids.shape

torch.Size([14])

Miért nem sikerült? "A 2. szakaszban a pipeline lépéseit követtük.

A probléma az, hogy egyetlen szekvenciát küldtünk a modellnek, miközben a  Transformers modellek alapértelmezésben több mondatot várnak el. Itt megpróbáltuk mindazt, amit a tokenizáló a színfalak mögött csinált, amikor egy szekvenciára alkalmaztuk. De ha jobban megnézzük, láthatjuk, hogy a tokenizáló nem egyszerűen csak átalakította a bemeneti azonosítók listáját egy tenzorrá, hanem egy dimenziót is hozzáadott hozzá:

In [ ]:
tokenized_inputs = tokenizer(sequence, return_tensors="pt")
print(tokenized_inputs["input_ids"])

tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102]])


Adjunk hozzá egy új dimenziót!

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for an NLP course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)

input_ids = torch.tensor([ids]) # itt adtuk hozzá az új dimenziót!!!
print("Input IDs:", input_ids)

output = model(input_ids)
print("Logits:", output.logits)

Input IDs: tensor([[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012]])
Logits: tensor([[-2.7276,  2.8789]], grad_fn=<AddmmBackward0>)


A kötegelés több mondat egyszerre történő elküldése a modellen keresztül. Ha csak egy mondatod van, akkor csak egyetlen szekvenciából építhetsz egy köteget:

Ez a tétel két azonos szekvenciából áll!

batched_ids = [ids, ids]

Próbáld ki! Konvertáld ezt a batched_ids listát tenzorrá, és add át a modelleden. Ellenőrizze, hogy ugyanazt a logitet adja, mint korábban (de kétszer)!

A kötegelés lehetővé teszi, hogy a modell akkor is működjön, ha több mondattal táplálod. Több szekvencia használata ugyanolyan egyszerű, mint egy szekvenciából álló batch létrehozása. Van azonban egy második probléma is. Amikor két (vagy több) mondatot próbál összefűzni, azok különböző hosszúságúak lehetnek. Ha dolgozott már tenzorokkal, akkor tudja, hogy azoknak téglalap alakúnak kell lenniük, így a bemeneti azonosítók listáját nem tudja közvetlenül tenzorrá alakítani. Ennek a problémának a kiküszöbölésére általában kitöltjük a bemeneteket.

A bemenetek kitöltése
A következő listát nem lehet tenzorrá alakítani:

In [ ]:
batched_ids = [
    [200, 200, 200],
    [200, 200]
]

Hogy ezt megkerüljük, a tenzorok téglalap alakúvá tételéhez kitöltést használunk. A kitöltés biztosítja, hogy minden mondatunk azonos hosszúságú legyen azáltal, hogy a kevesebb értékkel rendelkező mondatokhoz hozzáadunk egy speciális szót, a padding tokent. Ha például 10 mondatunk van 10 szóval és 1 mondatunk 20 szóval, a padding biztosítja, hogy minden mondat 20 szóval rendelkezzen. Példánkban az eredményül kapott tenzor így néz ki:

In [ ]:
padding_id = 100

batched_ids = [
    [200, 200, 200],
    [200, 200, padding_id],
]

A pad token azonosítója a tokenizer.pad_token_id mezőben található. Használjuk ezt, és küldjük át a két mondatunkat a modellen egyenként és összevontan:

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence1_ids = [[200, 200, 200]]
sequence2_ids = [[200, 200]]
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

print(model(torch.tensor(sequence1_ids)).logits)
print(model(torch.tensor(sequence2_ids)).logits)
print(model(torch.tensor(batched_ids)).logits)

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


tensor([[ 1.5694, -1.3895]], grad_fn=<AddmmBackward0>)
tensor([[ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)
tensor([[ 1.5694, -1.3895],
        [ 1.3374, -1.2163]], grad_fn=<AddmmBackward0>)


Valami baj van a logitokkal a kötegelt előrejelzéseinkben: a második sornak meg kellene egyeznie a második mondat logitjaival, de teljesen más értékeket kaptunk!

Ennek az az oka, hogy a Transformer-modellek legfontosabb jellemzője a figyelemrétegek, amelyek kontextusba helyezik az egyes tokeneket. Ezek figyelembe veszik a kitöltő tokeneket, mivel egy szekvencia összes tokenjére figyelnek. Ahhoz, hogy ugyanazt az eredményt kapjuk, amikor különböző hosszúságú egyes mondatokat engedünk át a modellen, vagy amikor egy köteget engedünk át ugyanazokkal a mondatokkal és kitöltéssel, meg kell mondanunk ezeknek a figyelemrétegeknek, hogy hagyják figyelmen kívül a kitöltő tokeneket. Ezt egy figyelemmaszk használatával érhetjük el.

##Figyelem maszkok

A figyelemmaszkok a bemeneti azonosítók tenzorával pontosan megegyező alakú tenzorok, amelyek 0-akkal és 1-ekkel vannak kitöltve: az 1-ek azt jelzik, hogy a megfelelő tokenekre figyelni kell, a 0-ak pedig azt, hogy a megfelelő tokenekre nem kell figyelni (azaz a modell figyelmi rétegei figyelmen kívül hagyják őket).

Egészítsük ki az előző példát egy figyelemmaszkkal:

In [ ]:
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

attention_mask = [
    [1, 1, 1],
    [1, 1, 0],
]

outputs = model(torch.tensor(batched_ids), attention_mask=torch.tensor(attention_mask))
print(outputs.logits)

tensor([[ 1.5694, -1.3895],
        [ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)


Most ugyanezeket a logiteket kapjuk a tétel második mondatára.

Figyeljük meg, hogy a második szekvencia utolsó értéke egy kitöltési azonosító, ami a figyelemmaszkban 0 érték.

✏️ Próbálja ki! Alkalmazza a tokenizálást kézzel a 2. pontban használt két mondatra ("Egész életemben egy HuggingFace-tanfolyamra vártam." és "Annyira utálom ezt!"). Vezesse át őket a modellen, és ellenőrizze, hogy ugyanazokat a logaritmusokat kapja-e, mint a 2. szakaszban. Most a padding token segítségével halmozza össze őket, majd hozza létre a megfelelő figyelemmaszkot. Ellenőrizze, hogy a modellen való áthaladáskor ugyanazokat az eredményeket kapja!

##Hosszabb szekvenciák
A transzformátor modellek esetében van egy korlát a szekvenciák hosszának, amelyeket át tudunk adni a modelleknek. A legtöbb modell 512 vagy 1024 tokenig terjedő szekvenciákat kezel, és összeomlik, ha hosszabb szekvenciák feldolgozására kérik. Erre a problémára két megoldás létezik:

Használjunk olyan modellt, amely hosszabb támogatott szekvenciahosszúságú.
Csonkítsa meg a szekvenciákat.

Ellenkező esetben a max_sequence_length paraméter megadásával csonkítsa meg a szekvenciákat:

In [ ]:
max_sequence_length = 256
sequence = sequence[:max_sequence_length]

Összeállítjuk az egészet


Azonban, ahogy a 2. szakaszban láttuk, a 🤗 Transformers API mindezt el tudja intézni helyettünk egy magas szintű függvénnyel, amelybe itt belemerülünk. Ha közvetlenül a mondaton hívjuk meg a tokenizálót, akkor olyan bemeneteket kapunk vissza, amelyek készen állnak arra, hogy átmenjenek a modellünkön:

In [ ]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

sequence = "I've been waiting for an NLP course my whole life."

model_inputs = tokenizer(sequence)

Itt a model_inputs változó tartalmazza mindazt, ami a modell jó működéséhez szükséges. A DistilBERT esetében ez magában foglalja a bemeneti azonosítókat, valamint a figyelmi maszkot. Más modellek, amelyek további bemeneteket fogadnak el, azokat is a tokenizáló objektum adja ki.

Amint azt az alábbi néhány példában látni fogjuk, ez a módszer nagyon hatékony. Először is, képes egyetlen szekvenciát tokenizálni:

In [ ]:
sequence = "I've been waiting for an NLP course my whole life."

model_inputs = tokenizer(sequence)

Egyszerre több szekvenciát is kezel, az API változatlanul hagyásával:

In [ ]:
sequences = ["I've been waiting for an NLP course my whole life.", "So have I!"]

model_inputs = tokenizer(sequences)

Többféle padding elérhető:

In [ ]:
# Will pad the sequences up to the maximum sequence length
model_inputs = tokenizer(sequences, padding="longest")

# Will pad the sequences up to the model max length
# (512 for BERT or DistilBERT)
model_inputs = tokenizer(sequences, padding="max_length")

# Will pad the sequences up to the specified max length
model_inputs = tokenizer(sequences, padding="max_length", max_length=8)

Többféle csonkolás érhető el:

In [ ]:
sequences = ["I've been waiting for an NLP course my whole life.", "So have I!"]

# Will truncate the sequences that are longer than the model max length
# (512 for BERT or DistilBERT)
model_inputs = tokenizer(sequences, truncation=True)

# Will truncate the sequences that are longer than the specified max length
model_inputs = tokenizer(sequences, max_length=8, truncation=True)

A tokenizáló objektum képes kezelni a konkrét keretrendszer-tenzorokká való átalakítást, amelyeket aztán közvetlenül elküldhetünk a modellnek. A következő kódpéldában például arra kérjük a tokenizert, hogy adja vissza a különböző keretrendszerek tenzorait - a "pt" a PyTorch tenzorokat, a "tf" a TensorFlow tenzorokat, az "np" pedig a NumPy tömböket adja vissza:

In [ ]:
sequences = ["I've been waiting for an NLP course my whole life.", "So have I!"]

# Returns PyTorch tensors
model_inputs = tokenizer(sequences, padding=True, return_tensors="pt")

# Returns TensorFlow tensors
model_inputs = tokenizer(sequences, padding=True, return_tensors="tf")

# Returns NumPy arrays
model_inputs = tokenizer(sequences, padding=True, return_tensors="np")

Különleges tokenek
Ha megnézzük a tokenizáló által visszaadott bemeneti azonosítókat, láthatjuk, hogy egy kicsit eltérnek a korábbiaktól:

In [ ]:
sequence = "I've been waiting for an NLP course my whole life."

model_inputs = tokenizer(sequence)
print(model_inputs["input_ids"])

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

[101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102]
[1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012]


Egy token azonosítót adtunk hozzá az elején, egyet pedig a végén. Dekódoljuk a fenti két ID-sorozatot, hogy lássuk, miről van szó:

In [ ]:
print(tokenizer.decode(model_inputs["input_ids"]))
print(tokenizer.decode(ids))

[CLS] i've been waiting for a huggingface course my whole life. [SEP]
i've been waiting for a huggingface course my whole life.


A tokenizáló hozzáadta a [CLS] speciális szót az elejére és a [SEP] speciális szót a végére. Ez azért van, mert a modell előzetesen ezekkel lett betanítva, így ahhoz, hogy a következtetésnél is ugyanolyan eredményt kapjunk, ezeket is hozzá kell adnunk. Megjegyezzük, hogy egyes modellek nem adnak hozzá speciális szavakat, vagy különböző szavakat adnak hozzá; a modellek ezeket a speciális szavakat is csak az elején vagy csak a végén adhatják hozzá. A tokenizáló mindenesetre tudja, hogy melyek az elvártak, és ezt elintézi helyettünk.

##Befejezés: A tokenizálótól a modellig
Most, hogy láttuk az összes egyes lépést, amit a tokenizáló objektum a szövegekre alkalmazva alkalmaz, nézzük meg még egyszer utoljára, hogyan tudja kezelni a többszörös szekvenciákat (kitöltés!), a nagyon hosszú szekvenciákat (csonkítás!) és a többféle tenzorokat a fő API-jával:

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = ["I've been waiting for an NLP course my whole life.", "So have I!"]

tokens = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")
output = model(**tokens)

In [ ]:
output.logits

tensor([[-1.5607,  1.6123],
        [-3.6183,  3.9137]], grad_fn=<AddmmBackward0>)